# 211011 get genome seqs

In [1]:
from pathlib import Path
import json
import sys
from urllib.request import urlretrieve

In [2]:
import pandas as pd
from Bio import Entrez
from tqdm.notebook import tqdm

In [3]:
from entrez_tools.misc import assembly_seq_url_from_esummary

In [4]:
sys.path.insert(0, './src')

from esummary_store import BasicEsummaryStore

## Setup

In [5]:
Entrez.email = 'mjlumpe@gmail.com'
Entrez.api_key = 'adea2a53e6e44c3cc42c932f9ba4bdffa809'

In [6]:
DATESTR = '211011'
NBNAME = DATESTR + '-get-genome-seqs'

In [7]:
infiles = dict(
    genomes=Path('data-processed/211011-get-genomes-list/211011-gambit-ani-additional-genomes.csv'),
    assembly_esummary=Path('/home/jared/projects/gambit/data/ncbi/assembly/esummary/')
)

In [8]:
tmpdir = Path('tmp')
tmpdir.mkdir(exist_ok=True)

In [9]:
genomes_dir = tmpdir / 'genomes'
genomes_dir.mkdir(exist_ok=True)

## Load data

In [10]:
genomes = pd.read_csv(infiles['genomes'], index_col=[0, 1])

In [11]:
accs = genomes['assembly_accession']

In [12]:
esummaries = BasicEsummaryStore('assembly', infiles['assembly_esummary'])

## Get FTP URLs

In [13]:
urls = dict()

for acc in accs:
    summary = esummaries.by_acc[acc]
    urls[acc] = assembly_seq_url_from_esummary(summary)

## Download seqs

In [14]:
seq_files = {acc: genomes_dir / f'{acc}.fasta.gz' for acc in accs}

In [15]:
_missing = [acc for acc, file in seq_files.items() if not file.is_file()]

for acc in tqdm(_missing):
    urlretrieve(urls[acc], seq_files[acc])
    file = seq_files[acc]

0it [00:00, ?it/s]